In [116]:
from pathlib import Path
from sqlalchemy import create_engine

import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

# Se define la ruta para la base de datos local SQLite
project_name = Path.cwd().parent.parent.name
data_projetc_path = Path.home().joinpath(f'Dropbox/La Casa Del Carpintero/Departamento de Programación/data_projects_git/data_{project_name}')
db_file = 'comisiones.db'
db_file_path_str = str(data_projetc_path.joinpath(db_file))

# Se crea el objeto engine para trabajarlo con los metodos de pandas
engine = create_engine(f'sqlite:///{db_file_path_str}')

In [117]:
file01 = 'clientes_regalos'
file01_path_str = str(data_projetc_path.joinpath(f'comisiones/{file01}.xlsx'))
clientes_regalos = pd.read_excel(file01_path_str, dtype_backend='numpy_nullable')

file02 = 'refacturaciones'
file02_path_str = str(data_projetc_path.joinpath(f'costo_ventas/{file02}.xlsx'))
refacturaciones = pd.read_excel(file02_path_str, dtype_backend='numpy_nullable')

In [118]:
with engine.connect() as conn, conn.begin():

    cobranza_año = pd.read_sql_table('cobranza_año_2024', conn, dtype_backend='numpy_nullable')

    costo_ventas_1_2024 = pd.read_sql_table('costo_ventas_1_2024', conn, dtype_backend='numpy_nullable')
    costo_ventas_2_2024 = pd.read_sql_table('costo_ventas_2_2024', conn, dtype_backend='numpy_nullable')
    costo_ventas_3_2024 = pd.read_sql_table('costo_ventas_3_2024', conn, dtype_backend='numpy_nullable')
    costo_ventas_4_2024 = pd.read_sql_table('costo_ventas_4_2024', conn, dtype_backend='numpy_nullable')
    costo_ventas_5_2024 = pd.read_sql_table('costo_ventas_5_2024', conn, dtype_backend='numpy_nullable')
    costo_ventas_6_2024 = pd.read_sql_table('costo_ventas_6_2024', conn, dtype_backend='numpy_nullable')
    costo_ventas_7_2024 = pd.read_sql_table('costo_ventas_7_2024', conn, dtype_backend='numpy_nullable')
    costo_ventas_8_2024 = pd.read_sql_table('costo_ventas_8_2024', conn, dtype_backend='numpy_nullable')
    
    datos_com_2023 = pd.read_sql_table('datos_com_2023', conn, dtype_backend='numpy_nullable')
    bonos_limites_comisiones = pd.read_sql_table('bonos_limites_comisiones', conn, dtype_backend='numpy_nullable')

engine.dispose()

costo_ventas_list = [
            costo_ventas_1_2024,
            costo_ventas_2_2024,
            costo_ventas_3_2024,
            costo_ventas_4_2024,
            costo_ventas_5_2024,
            costo_ventas_6_2024,
            costo_ventas_7_2024,
            costo_ventas_8_2024,
        ]

costo_ventas_año = pd.concat(costo_ventas_list)

In [119]:
def gerentes_list_func() -> pd.DataFrame:

    gerentes_list =  pd.DataFrame(
                [
                    [206, 'Mariana Araceli Carvajal Flores', 'Piso'],
                    [208, 'Carlos Manuel Guzmán Rios', 'CE'],
                    [209, 'Rosario Martinez Zarate', 'Piso'],
                ]
                ,
                columns = 
                    [
                        'salesperson_id',
                        'salesperson_name',
                        'business_model',
                    ]
            )

    return gerentes_list

In [120]:
def comisiones_mes_2024_func(mes:int, costo_ventas_list:list, bonos_limites_comisiones:pd.DataFrame, gerentes_list:pd.DataFrame) -> pd.DataFrame:

    df = costo_ventas_list[mes-1]
    mes_df = df['invoice_date'].dt.month.unique()[0]
    
    df.loc[:, 'mes'] = mes_df
    gerentes_list['mes'] = mes_df


    comisiones_2024_vend = (df.loc[
        ~df['salesperson_name'].isin(gerentes_list['salesperson_name'])
        ,
                [
                    'salesperson_id',
                    'salesperson_name',
                    'business_model',
                    'mes',
                    'price_subtotal',
                    'utilidad_partida_$_after_cc'
                ]
            ].groupby('salesperson_id')
            .agg({
                'salesperson_name':['first'], 
                'business_model':['first'], 
                'mes':['first'], 
                'price_subtotal':['sum'],
                'utilidad_partida_$_after_cc':['sum']
                })
            .set_axis(
                [
                    'salesperson_name',
                    'business_model',
                    'mes',
                    'facturacion',
                    'utilidad'
                ], axis='columns')
            .reset_index()
    )


    comisiones_2024_ger = gerentes_list.copy()

    comisiones_2024_ger_values = (df.loc[:,
                [
                    'sale_team_description',
                    'mes',
                    'price_subtotal',
                    'utilidad_partida_$_after_cc'
                ]
            ].groupby('sale_team_description')
            .agg({
                'mes':['first'], 
                'price_subtotal':['sum'],
                'utilidad_partida_$_after_cc':['sum']
                })
            .set_axis(
                [
                    'mes',
                    'facturacion',
                    'utilidad'
                ], axis='columns')
            .reset_index()
    )


    comisiones_2024_ger.loc[
            comisiones_2024_ger['salesperson_name'] == 'Carlos Manuel Guzmán Rios'
            , 
            ['facturacion', 'utilidad']
        ] = (comisiones_2024_ger_values[
                    comisiones_2024_ger_values['sale_team_description'].isin(['A1 CE', 'A2 CE'])
                ]
                .groupby('mes')
                .sum()
                [['facturacion', 'utilidad']]
                .values
            )

    comisiones_2024_ger.loc[
            comisiones_2024_ger['salesperson_name'] == 'Mariana Araceli Carvajal Flores'
            , 
            ['facturacion', 'utilidad']
        ] = comisiones_2024_ger_values[
                    comisiones_2024_ger_values['sale_team_description'] == 'A1 Piso'
                ][['facturacion', 'utilidad']].values

    comisiones_2024_ger.loc[
            comisiones_2024_ger['salesperson_name'] == 'Rosario Martinez Zarate'
            , 
            ['facturacion', 'utilidad']
        ] = comisiones_2024_ger_values[
                    comisiones_2024_ger_values['sale_team_description'] == 'A2 Piso'
                ][['facturacion', 'utilidad']].values


    comisiones_2024 = pd.concat([comisiones_2024_ger, comisiones_2024_vend], ignore_index=True)


    for i in range(len(comisiones_2024)):

        if comisiones_2024.iloc[i]['salesperson_name'] in gerentes_list['salesperson_name'].values:
            
            bonos_limites_comisiones_line = bonos_limites_comisiones.loc[
                (bonos_limites_comisiones['business_model'] == comisiones_2024.iloc[i]['business_model'])
                & (bonos_limites_comisiones['is_gerente'] == True)
                & (bonos_limites_comisiones['lim_inf'] <= comisiones_2024.iloc[i]['utilidad'])
                & (bonos_limites_comisiones['lim_sup'] >= comisiones_2024.iloc[i]['utilidad'])
            ]

        else:
            
            bonos_limites_comisiones_line = bonos_limites_comisiones.loc[
                (bonos_limites_comisiones['business_model'] == comisiones_2024.iloc[i]['business_model'])
                & (bonos_limites_comisiones['is_gerente'] == False)
                & (bonos_limites_comisiones['lim_inf'] <= comisiones_2024.iloc[i]['utilidad'])
                & (bonos_limites_comisiones['lim_sup'] >= comisiones_2024.iloc[i]['utilidad'])
            ]

            if bonos_limites_comisiones_line.empty:
                bonos_limites_comisiones_line = pd.DataFrame([[pd.NA, pd.NA]], columns = [ '%', 'bono'])

        comisiones_2024.loc[i, ['comision_%']] = bonos_limites_comisiones_line[['%']].iloc[0].values
        comisiones_2024.loc[i, ['comision_$']] = comisiones_2024.loc[i, ['comision_%']].iloc[0] * comisiones_2024.loc[i, ['utilidad']].iloc[0]
        comisiones_2024.loc[i, ['bono_venta_$']] = bonos_limites_comisiones_line[['bono']].iloc[0].values


    return comisiones_2024

In [121]:
def comision_cobranza_2024_func(cobranza_año:pd.DataFrame, datos_com_2023:pd.DataFrame, comisiones_2024:pd.DataFrame,  gerentes_list:pd.DataFrame) -> pd.DataFrame:
    
    cobranza_año.loc[:, ['fact_año']] = cobranza_año['fact_doc_date'].dt.year
    cobranza_año.loc[:, ['fact_mes']] = cobranza_año['fact_doc_date'].dt.month
    cobranza_año.loc[:, ['mes']] = cobranza_año['pay_date'].dt.month

    comision_cobranza_vend = cobranza_año.loc[
                                (cobranza_año['paga_comision'] == True)
                                & (~cobranza_año['salesperson_name'].isin(gerentes_list['salesperson_name']))
                                ,
                                [
                                    'salesperson_id',
                                    'salesperson_name',
                                    'mes',
                                    'pay_amount',
                                    'fact_año',
                                    'fact_mes'
                                ]
                            ].groupby(
                                ['salesperson_id', 
                                'salesperson_name', 
                                'mes', 
                                'fact_año', 
                                'fact_mes'
                                ]).sum().reset_index()


    comision_cobranza_ger = cobranza_año.loc[
                                (cobranza_año['paga_comision'] == True)
                                ,
                                [
                                    'sale_team_description',
                                    'mes',
                                    'pay_amount',
                                    'fact_año',
                                    'fact_mes'
                                ]
                            ].groupby(
                                ['sale_team_description', 
                                'mes', 
                                'fact_año', 
                                'fact_mes'
                                ]).sum().reset_index()


    comision_cobranza_ger.loc[
            comision_cobranza_ger['sale_team_description'].isin(['A1 CE', 'A2 CE'])
            ,
            ['salesperson_id', 'salesperson_name']
        ] = gerentes_list.loc[
                    gerentes_list['salesperson_name'] == 'Carlos Manuel Guzmán Rios'
                , 
                ['salesperson_id', 'salesperson_name']
            ].values

    comision_cobranza_ger.loc[
            comision_cobranza_ger['sale_team_description'] == 'A1 Piso'
            ,
            ['salesperson_id', 'salesperson_name']
        ] = gerentes_list.loc[
                    gerentes_list['salesperson_name'] == 'Mariana Araceli Carvajal Flores'
                , 
                ['salesperson_id', 'salesperson_name']
            ].values


    comision_cobranza_ger.loc[
            comision_cobranza_ger['sale_team_description'] == 'A2 Piso'
            ,
            ['salesperson_id', 'salesperson_name']
        ] = gerentes_list.loc[
                    gerentes_list['salesperson_name'] == 'Rosario Martinez Zarate'
                , 
                ['salesperson_id', 'salesperson_name']
            ].values

    comision_cobranza_ger['salesperson_id'] = comision_cobranza_ger['salesperson_id'].astype('Int64')


    comision_cobranza = pd.concat([comision_cobranza_vend, comision_cobranza_ger[comision_cobranza_vend.columns]], ignore_index=True)


    for i in range(len(comision_cobranza)):

        año = comision_cobranza.iloc[i]['fact_año']

        if año == 2022 or año == 2023:
            venta_comision_line = datos_com_2023.loc[
                                        (datos_com_2023['salesperson_id'] == comision_cobranza.iloc[i]['salesperson_id'])
                                        & (datos_com_2023['año'] == comision_cobranza.iloc[i]['fact_año'])
                                        & (datos_com_2023['mes'] == comision_cobranza.iloc[i]['fact_mes'])
                                    ]
        
        else:
            venta_comision_line = comisiones_2024.loc[
                                        (comisiones_2024['salesperson_id'] == comision_cobranza.iloc[i]['salesperson_id'])
                                        & (comisiones_2024['mes'] == comision_cobranza.iloc[i]['fact_mes'])
                                    ]
        
        comision_cobranza.loc[i, ['facturacion_c/iva']] = venta_comision_line[['facturacion']].iloc[0].values * 1.16
        comision_cobranza.loc[i, ['cobranza_%']] = (comision_cobranza.loc[i, ['pay_amount']].iloc[0] / comision_cobranza.loc[i, ['facturacion_c/iva']].iloc[0]) * 100
        comision_cobranza.loc[i, ['comision_mes']] = venta_comision_line[['comision_$']].iloc[0].values
        comision_cobranza.loc[i, ['comision_cobranza_$']] = comision_cobranza.loc[i, ['cobranza_%']].iloc[0] / 100 * comision_cobranza.loc[i, ['comision_mes']].iloc[0]

    return comision_cobranza

In [122]:
def comision_pagos_2024_func(comisiones_2024:pd.DataFrame, comision_cobranza_2024:pd.DataFrame) -> pd.DataFrame:

    comision_pagos = comision_cobranza_2024[[
                                'salesperson_id', 
                                'salesperson_name', 
                                'mes',
                                'comision_cobranza_$'
                            ]].groupby([
                                'salesperson_id', 
                                'salesperson_name', 
                                'mes', 
                            ]).sum().reset_index().merge(
                                comisiones_2024[[
                                    'salesperson_id', 
                                    'mes',
									'facturacion',
                                    'utilidad',
                                    'comision_%',
                                    'comision_$',
                                    'bono_venta_$',
                                ]],
                                how='left',
                                on=['salesperson_id', 'mes']
                            )

    comision_pagos.loc[
            ~comision_pagos[ 'bono_venta_$'].isna()
            ,
            'total_comision'
        ] = (comision_pagos['comision_cobranza_$'] + comision_pagos['bono_venta_$']) * 0.88

    comision_pagos.loc[
        comision_pagos[ 'bono_venta_$'].isna()
            ,
            'total_comision'
        ] = comision_pagos['comision_cobranza_$'] * 0.88


    return comision_pagos

In [123]:
gerentes_list = gerentes_list_func()

comisiones_1_2024 = comisiones_mes_2024_func(1, costo_ventas_list, bonos_limites_comisiones, gerentes_list)
comisiones_2_2024 = comisiones_mes_2024_func(2, costo_ventas_list, bonos_limites_comisiones, gerentes_list)
comisiones_3_2024 = comisiones_mes_2024_func(3, costo_ventas_list, bonos_limites_comisiones, gerentes_list)
comisiones_4_2024 = comisiones_mes_2024_func(4, costo_ventas_list, bonos_limites_comisiones, gerentes_list)
comisiones_5_2024 = comisiones_mes_2024_func(5, costo_ventas_list, bonos_limites_comisiones, gerentes_list)
comisiones_6_2024 = comisiones_mes_2024_func(6, costo_ventas_list, bonos_limites_comisiones, gerentes_list)
comisiones_7_2024 = comisiones_mes_2024_func(7, costo_ventas_list, bonos_limites_comisiones, gerentes_list)
comisiones_8_2024 = comisiones_mes_2024_func(8, costo_ventas_list, bonos_limites_comisiones, gerentes_list)

comisiones_2024 = pd.concat([
                        comisiones_1_2024,
                        comisiones_2_2024,
                        comisiones_3_2024,
                        comisiones_4_2024,
                        comisiones_5_2024,
                        comisiones_6_2024,
                        comisiones_7_2024,
                        comisiones_8_2024,
                    ])

comision_cobranza_2024 = comision_cobranza_2024_func(cobranza_año, datos_com_2023, comisiones_2024, gerentes_list)

comision_pagos_2024 = comision_pagos_2024_func(comisiones_2024, comision_cobranza_2024)

In [127]:
from IPython.display import display
    
id_vend = 208

display(comision_pagos_2024.loc[
        	comision_pagos_2024['salesperson_id'] == id_vend
            ,
				[
				'salesperson_id',
				'salesperson_name',
				'mes',
				'facturacion',
				'utilidad',
				'comision_%',
				'comision_$',
				'comision_cobranza_$',
				'bono_venta_$',
				'total_comision',
				]
			]
    )

display(comision_pagos_2024.loc[
        comision_pagos_2024['salesperson_id'] == id_vend
        ,
        ['salesperson_name', 'total_comision']
    ]
        .groupby('salesperson_name')
        .sum()
    )

,salesperson_id,salesperson_name,mes,facturacion,utilidad,comision_%,comision_$,comision_cobranza_$,bono_venta_$,total_comision
17,208,Carlos Manuel Guzmán Rios,1,"3,699,932.44","1,334,546.70",0.02,"26,690.93","19,199.23","25,000.00","38,895.32"
18,208,Carlos Manuel Guzmán Rios,2,"2,953,520.35","1,058,437.07",0.02,"21,168.74","18,944.71","20,000.00","34,271.35"
19,208,Carlos Manuel Guzmán Rios,3,"4,022,614.60","1,566,308.04",0.02,"31,326.16","26,479.04","30,000.00","49,701.55"
20,208,Carlos Manuel Guzmán Rios,4,"3,432,984.24","1,183,560.62",0.02,"23,671.21","23,146.09","20,000.00","37,968.56"
21,208,Carlos Manuel Guzmán Rios,5,"3,413,649.02","1,182,013.18",0.02,"23,640.26","21,483.79","20,000.00","36,505.73"
22,208,Carlos Manuel Guzmán Rios,6,"3,119,036.18","1,066,330.11",0.02,"21,326.60","23,007.66","20,000.00","37,846.74"
23,208,Carlos Manuel Guzmán Rios,7,"3,148,556.71","1,060,591.31",0.02,"21,211.83","24,692.03","20,000.00","39,328.99"
24,208,Carlos Manuel Guzmán Rios,8,"2,882,583.42","1,028,099.02",0.02,"20,561.98","22,748.04","20,000.00","37,618.27"


,total_comision
salesperson_name,
Carlos Manuel Guzmán Rios,"312,136.51"


In [125]:
# desktop_path = Path.home().joinpath('desktop')
# archivo_path = desktop_path.joinpath('comisiones_2024_ene-jul.xlsx')
# writer = pd.ExcelWriter(archivo_path, engine="openpyxl")

# comision_pagos_2024[['salesperson_id', 'salesperson_name']].groupby('salesperson_id').first().sort_values('salesperson_name').to_excel(writer, sheet_name='Vendedores')

# for id_vend in comision_pagos_2024['salesperson_id'].unique():

#     id_vend_mes = comision_pagos_2024.loc[
#         	comision_pagos_2024['salesperson_id'] == id_vend
#             ,
# 				[
# 				'salesperson_id',
# 				'salesperson_name',
# 				'mes',
# 				'facturacion',
# 				'utilidad',
# 				'comision_%',
# 				'comision_$',
# 				'comision_cobranza_$',
# 				'bono_venta_$',
# 				'total_comision',
# 				]
# 			]

#     id_vend_total = (comision_pagos_2024.loc[
#             comision_pagos_2024['salesperson_id'] == id_vend
#             ,
#             ['salesperson_name', 'total_comision']
#         ]
#             .groupby('salesperson_name')
#             .sum()
#         )


#     id_vend_mes.to_excel(writer, sheet_name=f'{id_vend}')
#     id_vend_total.to_excel(writer, sheet_name=f'{id_vend}', startrow=10)

# writer.close()
# writer.handles = None


# print('Se generó el archivo excel con éxito')

Se generó el archivo excel con éxito


In [126]:
# with engine.connect() as conn, conn.begin():  
#     comisiones_2024.to_sql(name = 'comisiones_2024', index= False, con = conn, if_exists = 'replace')
#     comision_cobranza_2024.to_sql(name = 'comision_cobranza_2024', index= False, con = conn, if_exists = 'replace')

# engine.dispose()

# print('Todas las tablas se ingresaron con éxito')

Todas las tablas se ingresaron con éxito
